In [1]:
#import sys
#sys.path.append('dba_utils/') # to access util files 

In [2]:
import numpy as np
import pandas as po
import matplotlib.pyplot as plt

In [3]:
!bash ./utils/build-cython.sh

rm: cannot remove '__pycache__': No such file or directory
Compiling dtw.pyx because it changed.
[1/1] Cythonizing dtw.pyx
running build_ext
building 'dtw' extension
creating build
creating build/temp.linux-x86_64-3.7
gcc -pthread -B /home/varun/anaconda3/envs/tf2/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include -I/home/varun/anaconda3/envs/tf2/include/python3.7m -c dtw.c -o build/temp.linux-x86_64-3.7/dtw.o
In file included from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832,
                 from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from dtw.c:619:
/home/varun/anaconda3/envs/tf2/lib/python3.7/

In [4]:
from utils.augment import augment_train_set

ModuleNotFoundError: No module named 'dba'

In [ ]:
df = po.read_csv('../data/processed/imputation/edtwbi.csv').sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_0 = df[df['FLAG'] == 0]
df_1 = df[df['FLAG'] == 1]

In [ ]:
# Undersampling
df_0 = df_0.sort_values('num_zeros')[:len(df_1)]

In [ ]:
df_undersampled = po.concat([df_1, df_0], axis=0, ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_undersampled['FLAG'].value_counts()

In [ ]:
X_undersampled = df_undersampled.drop(['num_zeros', 'FLAG'], axis=1)
y_undersampled = df_undersampled['FLAG']

In [ ]:
X_undersampled_train = X_undersampled[:int(0.6*len(X_undersampled))]
y_undersampled_train = y_undersampled[:int(0.6*len(X_undersampled))]

X_undersampled_val   = X_undersampled[int(0.6*len(X_undersampled)):int(0.8*len(X_undersampled))]
y_undersampled_val   = y_undersampled[int(0.6*len(X_undersampled)):int(0.8*len(X_undersampled))]

X_undersampled_test  = X_undersampled[int(0.8*len(X_undersampled)):]
y_undersampled_test  = y_undersampled[int(0.8*len(X_undersampled)):]

In [ ]:
X_undersampled_train     = X_undersampled_train.to_numpy()[:, :, np.newaxis]
X_undersampled_val       = X_undersampled_val.to_numpy()[:, :, np.newaxis]
X_undersampled_test      = X_undersampled_test.to_numpy()[:, :, np.newaxis]

In [ ]:
y_undersampled_train     = y_undersampled_train.to_numpy()
y_undersampled_val       = y_undersampled_val.to_numpy()
y_undersampled_test      = y_undersampled_test.to_numpy()

In [ ]:
# Save undersampled data
with open('../data/processed/training_data/undersampled.npz', 'wb') as f:
    np.savez(f, X_train=X_undersampled_train, y_train=y_undersampled_train, X_val=X_undersampled_val, y_val=y_undersampled_val, X_test=X_undersampled_test, y_test=y_undersampled_test)

In [ ]:
X = df.drop(['num_zeros', 'FLAG'], axis=1)
y = df['FLAG']

In [ ]:
X_train = X[:int(0.6*len(X))]
y_train = y[:int(0.6*len(X))]

X_val   = X[int(0.6*len(X)):int(0.8*len(X))]
y_val   = y[int(0.6*len(X)):int(0.8*len(X))]

X_test  = X[int(0.8*len(X)):]
y_test  = y[int(0.8*len(X)):]

In [17]:
y_train.value_counts()

0    12381
1     2182
Name: FLAG, dtype: int64

In [18]:
y_val.value_counts()

0    4121
1     733
Name: FLAG, dtype: int64

In [19]:
y_test.value_counts()

0    4155
1     700
Name: FLAG, dtype: int64

In [20]:
X_train     = X_train.to_numpy()[:, :, np.newaxis]
X_val       = X_val.to_numpy()[:, :, np.newaxis]
X_test      = X_test.to_numpy()[:, :, np.newaxis]

In [21]:
y_train     = y_train.to_numpy()
y_val       = y_val.to_numpy()
y_test      = y_test.to_numpy()

In [22]:
# Save raw data
with open('../data/processed/training_data/standard.npz', 'wb') as f:
    np.savez(f, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)

In [23]:
classes = 2
N       = 100 

In [24]:
np.bincount(y_train)

array([12381,  2182])

In [25]:
# augment the dataset
X_train_synth, y_train_synth = augment_train_set(X_train, y_train, classes, N, limit_N = False, weights_method_name='as', distance_algorithm='dtw')

NameError: name 'augment_train_set' is not defined

In [11]:
            
                syn_train_set, distance_algorithm = augment_function('as_dtw_dba_augment',
                                                                     x_train, y_train, classes,
                                                                     nb_prototypes, limit_N=False)
                # get the synthetic train and labels
                syn_x_train, syn_y_train = syn_train_set
                # concat the synthetic with the reduced random train and labels
                aug_x_train = np.array(x_train.tolist() + syn_x_train.tolist())
                aug_y_train = np.array(y_train.tolist() + syn_y_train.tolist())

                print(np.unique(y_train,return_counts=True))
                print(np.unique(aug_y_train,return_counts=True))

                y_pred = classifier.fit(aug_x_train, aug_y_train, x_test,
                                        y_test)